In [1]:
import re
import pandas as pd
import random
import numpy as np

filename = "C:\\Users\\mustafa\\Documents\\test.txt"
filename2 = "C:\\Users\\mustafa\\Documents\\test2.txt"
filename3 = "C:\\Users\\mustafa\\Documents\\test3.txt"

filenames = [filename, filename2, filename3]

In [2]:
def extract_all_shingles(filenames):
    
    k4_dict = {}
    k5_dict = {}
    k8_dict = {}
    
    for filename in filenames:
        
        file = open(filename, "r") 
        text = file.read()
        text = re.sub('[^0-9a-zA-Z]+', ' ', text)
        text = re.sub('  ', ' ', text)
        text = text.lower()

        for i in range(len(text)):
            k4 = text[i : i+4]
            k5 = text[i : i+5]
            k8 = text[i : i+8]

            if k4 not in k4_dict:
                k4_dict[k4] = 0
            if k5 not in k5_dict:
                k5_dict[k5] = 0
            if k8 not in k8_dict:
                k8_dict[k8] = 0

    return k4_dict, k5_dict, k8_dict   

    
def shingle(filename, k4_dict, k5_dict, k8_dict):
      
    k4_dict1 = k4_dict.copy()
    k5_dict1 = k5_dict.copy()
    k8_dict1 = k8_dict.copy()
                
    file = open(filename, "r") 
    text = file.read()
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = re.sub('  ', ' ', text)
    text = text.lower()

    for i in range(len(text)):
        k4 = text[i : i+4]
        k5 = text[i : i+5]
        k8 = text[i : i+8]

        if k4 in k4_dict1:
            k4_dict1[k4] = 1
        if k5 in k5_dict1:
            k5_dict1[k5] = 1
        if k8 in k8_dict1:
            k8_dict1[k8] = 1

    return k4_dict1, k5_dict1, k8_dict1     


def jaccard(x, y):
    T1 = x == 1
    T2 = y == 1

    F1 = x == 0
    F2 = y == 0

    total = len(x)
    ratio = np.sum(T1 == T2) / (total - np.sum(F1 == F2))
    return ratio
            
    
def hash(a):
    def sub_hash(x, m = 991):
        return (a*x + 1) % m
    return sub_hash

In [3]:
k4_dict, k5_dict, k8_dict = extract_all_shingles(filenames)

k4_matrix = np.zeros((len(k4_dict), len(filenames)))
k5_matrix = np.zeros((len(k5_dict), len(filenames)))
k8_matrix = np.zeros((len(k8_dict), len(filenames)))

for i, filename in enumerate(filenames):
    k4_dict1, k5_dict1, k8_dict1 = shingle(filename, k4_dict, k5_dict, k8_dict)
    k4_matrix[:,i] = np.fromiter(k4_dict1.values(), dtype = float)
    k5_matrix[:,i] = np.fromiter(k5_dict1.values(), dtype = float)
    k8_matrix[:,i] = np.fromiter(k8_dict1.values(), dtype = float)

In [4]:
n = len(filenames)
k_shingle_matrix = np.zeros((n, 3))

for i in range(len(filenames)):
    k_shingle_matrix[i, 0] = np.sum(k4_matrix[:,i])
    k_shingle_matrix[i, 1] = np.sum(k5_matrix[:,i])
    k_shingle_matrix[i, 2] = np.sum(k8_matrix[:,i])

k_shingle_matrix = pd.DataFrame(k_shingle_matrix)
k_shingle_matrix.index = filenames
k_shingle_matrix.columns = ["k4", "k5", "k8"]

print("Hangi dökümanda kaç adet K Shingle olduğunu gösteren matris")
k_shingle_matrix

Hangi dökümanda kaç adet K Shingle olduğunu gösteren matris


,k4,k5,k8
C:\Users\mustafa\Documents\test.txt,190.0,198.0,209.0
C:\Users\mustafa\Documents\test2.txt,156.0,165.0,179.0
C:\Users\mustafa\Documents\test3.txt,231.0,250.0,280.0


In [5]:
print("k4 için toplam shingle sayısı " + str(k4_matrix.shape[0]))
print("k5 için toplam shingle sayısı " + str(k5_matrix.shape[0]))
print("k8 için toplam shingle sayısı " + str(k8_matrix.shape[0]))

k4 için toplam shingle sayısı 523
k5 için toplam shingle sayısı 577
k8 için toplam shingle sayısı 662


In [6]:
def calc_shingle_similarity(k_matrix):
    # k matrisinin kolon sayısı döküman sayısını içermekte
    n = k_matrix.shape[1]
    shingle_similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        shingle_similarity_matrix[i, i] = 1
        for j in range(i):
            shingle_similarity_matrix[i, j] = jaccard(k_matrix[:, i], k_matrix[:, j])
            shingle_similarity_matrix[j, i] = shingle_similarity_matrix[i, j]

    return shingle_similarity_matrix


n = len(filenames)

shingle_similarity_matrix_k4 = np.zeros((n, n))
shingle_similarity_matrix_k5 = np.zeros((n, n))
shingle_similarity_matrix_k8 = np.zeros((n, n))

shingle_similarity_matrix_k4 = calc_shingle_similarity(k4_matrix)
shingle_similarity_matrix_k5 = calc_shingle_similarity(k5_matrix)
shingle_similarity_matrix_k8 = calc_shingle_similarity(k8_matrix)

print("k4 shingle similarity matrisi")
print(shingle_similarity_matrix_k4)
print("")

print("k5 shingle similarity matrisi")
print(shingle_similarity_matrix_k5)
print("")

print("k8 shingle similarity matrisi")
print(shingle_similarity_matrix_k8)
print("")

k4 shingle similarity matrisi
[[ 1.          0.82867133  0.3307888 ]
 [ 0.82867133  1.          0.53372434]
 [ 0.3307888   0.53372434  1.        ]]

k5 shingle similarity matrisi
[[ 1.          0.77538462  0.33564815]
 [ 0.77538462  1.          0.5144357 ]
 [ 0.33564815  0.5144357   1.        ]]

k8 shingle similarity matrisi
[[ 1.          0.72395833  0.35378323]
 [ 0.72395833  1.          0.46784922]
 [ 0.35378323  0.46784922  1.        ]]



In [7]:
#hashx 100 adet hash fonksiyonunu tutuyor.
hashx = []
rand_hash = random.sample(range(1, 991), 100)

for i in range(100):
    hashx.append(hash(rand_hash[i]))

In [8]:
shingle_size = len(k4_dict)

nrow = shingle_size
ncol = len(filenames)
nhash = len(hashx)


def calc_signature_matrix(matris):
    
    matris_hash = np.zeros((nrow, nhash))
    signature_matrix = np.full((nhash, ncol), 9999999)
    
    for i in range(matris_hash.shape[0]):
        for j in range(matris_hash.shape[1]):
            h = hashx[j]
            matris_hash[i, j] = h(i + 1)

        for j in range(matris.shape[1]):
            if(matris[i, j] == 1):
                for k in range(matris_hash.shape[1]):
                    if(signature_matrix[k, j] > matris_hash[i, k]):
                        signature_matrix[k, j] = matris_hash[i, k]
    
    return(signature_matrix)

k4_signature_matrix = calc_signature_matrix(k4_matrix)
k5_signature_matrix = calc_signature_matrix(k5_matrix)
k8_signature_matrix = calc_signature_matrix(k8_matrix)            

In [9]:
def calc_signature_similarity(sig_matrix):
    n = sig_matrix.shape[1]
    signature_similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        signature_similarity_matrix[i, i] = 1
        for j in range(i):
            signature_similarity_matrix[i, j] = jaccard_signature(sig_matrix[:, i], sig_matrix[:, j])
            signature_similarity_matrix[j, i] = signature_similarity_matrix[i, j]

    return signature_similarity_matrix


def jaccard_signature(x, y):
    return np.sum(x == y) / len(x)

In [10]:
n = len(filenames)

signature_similarity_matrix_k4 = np.zeros((n, n))
signature_similarity_matrix_k5 = np.zeros((n, n))
signature_similarity_matrix_k8 = np.zeros((n, n))

signature_similarity_matrix_k4 = calc_signature_similarity(k4_signature_matrix)
signature_similarity_matrix_k5 = calc_signature_similarity(k5_signature_matrix)
signature_similarity_matrix_k8 = calc_signature_similarity(k8_signature_matrix)

print("k4 signature similarity matrisi")
print(signature_similarity_matrix_k4)

print("k5 signature similarity matrisi")
print(signature_similarity_matrix_k5)

print("k8 signature similarity matrisi")
print(signature_similarity_matrix_k8)

k4 signature similarity matrisi
[[ 1.    0.09  0.03]
 [ 0.09  1.    0.14]
 [ 0.03  0.14  1.  ]]
k5 signature similarity matrisi
[[ 1.    0.01  0.  ]
 [ 0.01  1.    0.04]
 [ 0.    0.04  1.  ]]
k8 signature similarity matrisi
[[ 1.    0.02  0.  ]
 [ 0.02  1.    0.02]
 [ 0.    0.02  1.  ]]
